# Etapa 3 - Limpieza de Datos

**Proyecto:** Ciencia de Datos - Preparación de Datos  
**Universidad:** Pontificia Universidad Javeriana  
**Curso:** Tecnologías Emergentes 2025  
**Profesor:** Luis Carlos Chicaíza

---

## Objetivo de esta Etapa

Realizar la **limpieza sistemática** del dataset de accidentes de tránsito en Bucaramanga, abordando:

1. Reconocimiento y tratamiento de atributos con valores únicos o casi únicos.  
2. Análisis y tratamiento de valores faltantes.  
3. Identificación y tratamiento de valores atípicos o inconsistentes.  
4. Generación de un dataset limpio y documentado para la etapa 4 (vista minable).


## 1. Configuración Inicial


In [15]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import sys
import os
sys.path.append(os.path.abspath(".."))

from src.limpieza_datos import (
    cargar_datos,
    diagnostico_calidad,
    tratar_atributos_unicos,
    tratar_valores_faltantes,
    tratar_atipicos_iqr,
    normalizar_texto_categorico
)

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Configuración de visualización
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas y configuradas")

✓ Librerías importadas y configuradas


## 2. Carga del Dataset y Diagnóstico Inicial


In [16]:
# Cargar dataset original (mismas rutas usadas en las etapas 1 y 2)
file_path = '../data/raw/accidentes_transito.csv'

# Intentar cargar con diferentes encodings si es necesario
try:
    df_raw = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df_raw = pd.read_csv(file_path, encoding='latin-1')

print("=" * 80)
print("DATASET ORIGINAL - INFORMACIÓN BÁSICA")
print("=" * 80)
print(f"Registros: {len(df_raw):,}")
print(f"Atributos: {len(df_raw.columns)}")
print("\nPrimeras filas:")
display(df_raw.head())

print("\nInformación general:")
print(df_raw.info())


DATASET ORIGINAL - INFORMACIÓN BÁSICA
Registros: 39,193
Atributos: 24

Primeras filas:


,ORDEN,FECHA,AÑO,MES,DÍA,GRAVEDAD,PEATON,AUTOMOVIL,CAMPERO,CAMIONETA,MICRO,BUSETA,BUS,CAMION,VOLQUETA,MOTO,BICICLETA,OTRO,BARRIO,HORA,ENTIDAD,COMUNA,Propietario de Vehículo,DIURNIO/NOCTURNO
0,1,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Con heridos,0,1,0,0,0,0,0,0,0,0,0,0,Mutis,1899-12-31T12:15:00.000,AGENTES DTB,17. MUTIS,Particular,Diurno
1,2,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,1,0,1,0,0,0,0,0,0,0,0,Regaderos Norte,1899-12-31T14:00:00.000,AGENTES DTB,02. NORORIENTAL,Empresa,Diurno
2,3,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,0,0,1,0,0,0,0,0,0,0,0,Cabecera Del Llano,1899-12-31T12:00:00.000,AGENTES DTB,12. CABECERA DEL LLANO,Particular,Diurno
3,4,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Solo daños,0,1,0,1,0,0,0,0,0,0,0,0,Norte Bajo,1899-12-31T18:30:00.000,AGENTES DTB,03. SAN FRANCISCO,Empresa,Nocturno
4,5,2012-01-01T00:00:00.000,2012,01. Enero,07. Domingo,Con heridos,1,0,0,0,0,0,0,0,0,1,0,0,Dangond,1899-12-31T00:30:00.000,AGENTES DTB,11. SUR,Particular,Nocturno



Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39193 entries, 0 to 39192
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ORDEN                    39193 non-null  int64 
 1   FECHA                    39193 non-null  object
 2   AÑO                      39193 non-null  int64 
 3   MES                      39193 non-null  object
 4   DÍA                      39193 non-null  object
 5   GRAVEDAD                 39193 non-null  object
 6   PEATON                   39193 non-null  int64 
 7   AUTOMOVIL                39193 non-null  int64 
 8   CAMPERO                  39193 non-null  int64 
 9   CAMIONETA                39193 non-null  int64 
 10  MICRO                    39193 non-null  int64 
 11  BUSETA                   39193 non-null  int64 
 12  BUS                      39193 non-null  int64 
 13  CAMION                   39193 non-null  int64 
 14  VOLQUETA        

### 2.1 Tabla de Diagnóstico de Calidad de Datos

Como primer paso, se construye una tabla de diagnóstico por atributo, que resume:

- Tipo de dato.  
- Número y porcentaje de valores faltantes.  
- Cardinalidad (número de valores distintos).  
- Ejemplos de valores observados.

Esto permite identificar rápidamente columnas con muchos nulos, alta unicidad o posibles problemas de calidad.


In [17]:
# Construir tabla de diagnóstico usando la función auxiliar
tabla_diagnostico = diagnostico_calidad(df_raw)

print("=" * 80)
print("TABLA DE DIAGNÓSTICO DE CALIDAD - VISTA GENERAL")
print("=" * 80)
display(tabla_diagnostico)

print("\nResumen de valores faltantes por columna:")
cols_con_nulos = tabla_diagnostico[tabla_diagnostico['n_nulos'] > 0]
if not cols_con_nulos.empty:
    for _, row in cols_con_nulos.iterrows():
        print(f" - {row['columna']}: {row['n_nulos']:,} nulos ({row['pct_nulos']}%)")
else:
    print("No se encontraron valores nulos en las columnas analizadas.")


TABLA DE DIAGNÓSTICO DE CALIDAD - VISTA GENERAL


,columna,tipo,n_nulos,pct_nulos,cardinalidad,ejemplo_valores
0,ORDEN,int64,0,0.00,39193,"[1, 2, 3]"
1,FECHA,object,0,0.00,4264,"[2012-01-01T00:00:00.000, 2012-01-02T00:00:00...."
2,AÑO,int64,0,0.00,12,"[2012, 2014, 2016]"
3,MES,object,0,0.00,12,"[01. Enero, 02. Febrero, 03. Marzo]"
4,DÍA,object,0,0.00,7,"[07. Domingo, 01. Lunes, 02. Martes]"
5,GRAVEDAD,object,0,0.00,4,"[Con heridos, Solo daños, Con muertos]"
6,PEATON,int64,0,0.00,8,"[0, 1, 2]"
7,AUTOMOVIL,int64,0,0.00,10,"[1, 0, 2]"
8,CAMPERO,int64,0,0.00,3,"[0, 1, 2]"
9,CAMIONETA,int64,0,0.00,4,"[0, 1, 2]"



Resumen de valores faltantes por columna:
No se encontraron valores nulos en las columnas analizadas.


## 3. Tratamiento de Atributos con Valores Únicos o Casi Únicos

En esta sección se identifican columnas que:

- Funcionan como identificadores puros (por ejemplo, `ORDEN`).  
- Tienen una **cardinalidad muy alta** (casi un valor distinto por fila).

Las columnas que actúan como ID se eliminan del análisis; las de alta unicidad se documentan y se decide si se conservan por su relevancia analítica.

In [18]:
# Aplicar tratamiento de atributos únicos / casi únicos
df_unicos, decisiones_unicos = tratar_atributos_unicos(df_raw)

print("=" * 80)
print("TRATAMIENTO DE ATRIBUTOS ÚNICOS / CASI ÚNICOS")
print("=" * 80)
print("Decisiones tomadas:")
for col, decision in decisiones_unicos.items():
    print(f" - {col}: {decision}")

print("\nForma del dataset antes / después:")
print(f" - Original: {df_raw.shape[0]:,} filas, {df_raw.shape[1]} columnas")
print(f" - Tras tratamiento de unicidad: {df_unicos.shape[0]:,} filas, {df_unicos.shape[1]} columnas")


TRATAMIENTO DE ATRIBUTOS ÚNICOS / CASI ÚNICOS
Decisiones tomadas:
 - ORDEN: eliminada (columna ID explícita)

Forma del dataset antes / después:
 - Original: 39,193 filas, 24 columnas
 - Tras tratamiento de unicidad: 39,193 filas, 23 columnas


## 4. Tratamiento de Valores Faltantes

A partir de la tabla de diagnóstico, se definen criterios para el manejo de nulos:

- Columnas con más del **50% de valores faltantes** se eliminan.  
- En el resto:
  - Variables numéricas: imputación con la **mediana**.  
  - Variables categóricas: imputación con la **moda** (valor más frecuente).

Este criterio busca un balance entre conservar información y evitar sesgos excesivos por imputación.


In [19]:
# Aplicar tratamiento de valores faltantes
df_nulos, decisiones_nulos = tratar_valores_faltantes(df_unicos)

print("=" * 80)
print("TRATAMIENTO DE VALORES FALTANTES")
print("=" * 80)

print("Decisiones por columna (muestra):")
for col, decision in list(decisiones_nulos.items())[:30]:
    print(f" - {col}: {decision}")

print("\nNúmero de columnas eliminadas por alto porcentaje de nulos:")
eliminadas = [c for c, d in decisiones_nulos.items() if 'eliminada' in d]
print(f" - Total columnas eliminadas: {len(eliminadas)}")
if eliminadas:
    print("   Columnas eliminadas:", ', '.join(eliminadas))

print("\nForma del dataset tras tratamiento de nulos:")
print(f" - Filas: {df_nulos.shape[0]:,}")
print(f" - Columnas: {df_nulos.shape[1]}")


TRATAMIENTO DE VALORES FALTANTES
Decisiones por columna (muestra):
 - FECHA: sin nulos
 - AÑO: sin nulos
 - MES: sin nulos
 - DÍA: sin nulos
 - GRAVEDAD: sin nulos
 - PEATON: sin nulos
 - AUTOMOVIL: sin nulos
 - CAMPERO: sin nulos
 - CAMIONETA: sin nulos
 - MICRO: sin nulos
 - BUSETA: sin nulos
 - BUS: sin nulos
 - CAMION: sin nulos
 - VOLQUETA: sin nulos
 - MOTO: sin nulos
 - BICICLETA: sin nulos
 - OTRO: sin nulos
 - BARRIO: sin nulos
 - HORA: sin nulos
 - ENTIDAD: sin nulos
 - COMUNA: sin nulos
 - Propietario de Vehículo: sin nulos
 - DIURNIO/NOCTURNO: sin nulos

Número de columnas eliminadas por alto porcentaje de nulos:
 - Total columnas eliminadas: 0

Forma del dataset tras tratamiento de nulos:
 - Filas: 39,193
 - Columnas: 23


## 5. Normalización de Variables Categóricas

Se realiza una limpieza básica de texto en variables categóricas clave:

- Eliminación de espacios en blanco al inicio y al final.  
- Unificación de espacios intermedios múltiples.

Esto ayuda a reducir problemas derivados de valores como `" Norte "` vs `"Norte"` o variaciones similares.


In [20]:
# Definir columnas categóricas a normalizar (según exploración previa)
columnas_texto = [
    "BARRIO",
    "COMUNA",
    "GRAVEDAD",
    "MES",
    "DÍA",
    "ENTIDAD",
    "Propietario de Vehículo",
    "DIURNIO/NOCTURNO",
]

df_cat = normalizar_texto_categorico(df_nulos, columnas_texto)

print("=" * 80)
print("NORMALIZACIÓN DE VARIABLES CATEGÓRICAS")
print("=" * 80)
print("Columnas normalizadas:")
for col in columnas_texto:
    if col in df_cat.columns:
        print(f" - {col}")
    else:
        print(f" - {col} (no presente en el DataFrame tras pasos anteriores)")

# Normalización específica de la variable GRAVEDAD
if "GRAVEDAD" in df_cat.columns:
    df_cat["GRAVEDAD"] = (
        df_cat["GRAVEDAD"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.lower()
        .map({
            "solo daños": "Solo daños",
            "solo danos": "Solo daños",
            "con heridos": "Con heridos",
            "con muertos": "Con muertos",
        })
        .fillna("Otra")
    )

# Asignar el DataFrame normalizado a df_limpio
df_limpio = df_cat


NORMALIZACIÓN DE VARIABLES CATEGÓRICAS
Columnas normalizadas:
 - BARRIO
 - COMUNA
 - GRAVEDAD
 - MES
 - DÍA
 - ENTIDAD
 - Propietario de Vehículo
 - DIURNIO/NOCTURNO


## 6. Tratamiento de Valores Atípicos (Outliers) en Variables Numéricas

Para las variables numéricas se aplican reglas basadas en el **rango intercuartílico (IQR)**:

- Se calculan Q1, Q3 e IQR = Q3 − Q1.  
- Se fijan límites inferior y superior:  
  \\( \\text{Límite Inf} = Q1 - 1.5 \\times IQR \\),  
  \\( \\text{Límite Sup} = Q3 + 1.5 \\times IQR \\).  
- Los valores que quedan por fuera se **recortan** (winsorización) a dichos límites.

Con esto se reduce el impacto de registros extremos sin eliminarlos por completo.


In [21]:
# Aplicar tratamiento de valores atípicos numéricos
df_limpio, resumen_atipicos = tratar_atipicos_iqr(df_cat)

print("=" * 80)
print("TRATAMIENTO DE VALORES ATÍPICOS NUMÉRICOS (IQR)")
print("=" * 80)

if resumen_atipicos:
    resumen_df = pd.DataFrame.from_dict(resumen_atipicos, orient='index')
    display(resumen_df)
    print("\nColumnas con valores modificados (n_modificados > 0):")
    modificadas = resumen_df[resumen_df['n_modificados'] > 0]
    if not modificadas.empty:
        for col, row in modificadas.iterrows():
            print(f" - {col}: {int(row['n_modificados'])} registros recortados")
    else:
        print("No se detectaron outliers según el criterio IQR.")
else:
    print("No se generó resumen de atípicos (posiblemente no hay columnas numéricas con variación suficiente).")

print("\nForma final del dataset limpio:")
print(f" - Filas: {df_limpio.shape[0]:,}")
print(f" - Columnas: {df_limpio.shape[1]}")


TRATAMIENTO DE VALORES ATÍPICOS NUMÉRICOS (IQR)


,q1,q3,lim_inf,lim_sup,n_modificados
AÑO,2014.00,2019.00,2006.50,2026.50,0
AUTOMOVIL,0.00,1.00,-1.50,2.50,360
MOTO,0.00,1.00,-1.50,2.50,107



Columnas con valores modificados (n_modificados > 0):
 - AUTOMOVIL: 360 registros recortados
 - MOTO: 107 registros recortados

Forma final del dataset limpio:
 - Filas: 39,193
 - Columnas: 23


## 7. Exportación del Dataset Limpio

Finalmente, se guarda el resultado de la limpieza en la carpeta `data/processed/`, para su uso en la etapa 4 (construcción de vista minable y análisis posterior).


In [22]:
# Guardar dataset limpio
output_path = '../data/processed/accidentes_transito_limpio.csv'
import os

os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_limpio.to_csv(output_path, index=False, encoding='utf-8')

print("=" * 80)
print("EXPORTACIÓN DEL DATASET LIMPIO")
print("=" * 80)
print(f"Archivo guardado en: {output_path}")


EXPORTACIÓN DEL DATASET LIMPIO
Archivo guardado en: ../data/processed/accidentes_transito_limpio.csv


## 8. Conclusiones de la Etapa 3 - Limpieza de Datos

### ✅ Resumen de Transformaciones Clave

- **Atributos únicos / casi únicos**  
  - Se eliminó la columna de identificación `ORDEN`, al no aportar información analítica.  
  - Se documentaron columnas de alta unicidad (por ejemplo, fechas) que se conservaron por su relevancia temporal.

- **Valores faltantes**  
  - Se eliminaron columnas con más del 50% de valores faltantes (criterio de baja confiabilidad).  
  - En el resto de atributos:
    - Variables numéricas: imputación con la mediana.  
    - Variables categóricas: imputación con la moda o un valor marcador (`SIN_DATO`) cuando fue necesario.

- **Valores atípicos**  
  - Se aplicó el método IQR con winsorización en variables numéricas, reduciendo el impacto de registros extremos sin eliminar observaciones completas.

- **Normalización de texto**  
  - Se unificaron formatos en variables categóricas clave (`GRAVEDAD`, `BARRIO`, `COMUNA`, `DIURNIO/NOCTURNO`, entre otras), reduciendo la dispersión artificial de categorías por diferencias de escritura.

### 🎯 Impacto en el Proyecto

- El dataset resultante presenta **mejor consistencia y calidad**, lo cual:
  - Facilita la interpretación de patrones en las etapas siguientes.  
  - Reduce el riesgo de conclusiones sesgadas por errores de captura o datos extremos.  
  - Deja documentadas las decisiones de limpieza, lo que aporta transparencia y reproducibilidad al proyecto.

Con esta etapa, se completa el macroproceso de **limpieza de datos**, dejando el dataset listo para la construcción de la **vista minable** y el análisis más profundo en la Etapa 4.
